### Ejercicio 19: Blending

In [27]:
from pyomo.environ import *
import numpy as np
import pandas as pd
model = ConcreteModel()

In [28]:
df = pd.DataFrame({'MaxDisp': [3000,2000,4000,1000], 'Coste': [13,15.3,14.6,14.9]}, index=[1,2,3,4])
df2 = pd.DataFrame({'Precio': [16.2, 15.75, 15.3]}, index=['A', 'B', 'C'])

model.c = Set(initialize=df.index)
model.g = Set(initialize=df2.index)

model.F = Var(model.c,model.g, domain= NonNegativeReals)

In [29]:
model.A1 = Constraint(expr=model.F[1,'A']-0.15*sum(model.F[i,'A'] for i in model.c)<=0)
model.A2 = Constraint(expr=model.F[2,'A']-0.4*sum(model.F[i,'A'] for i in model.c)>=0)
model.A3 = Constraint(expr=model.F[3,'A']-0.5*sum(model.F[i,'A'] for i in model.c)<=0)
model.B1 = Constraint(expr=model.F[1,'B']-0.1*sum(model.F[i,'B'] for i in model.c)<=0)
model.B2 = Constraint(expr=model.F[2,'B']-0.1*sum(model.F[i,'B'] for i in model.c)<=0)
model.C1 = Constraint(expr=model.F[1,'C']-0.2*sum(model.F[i,'C'] for i in model.c)<=0)

model.disp = ConstraintList()
for i in model.c:
    model.disp.add(
        sum(model.F[i,j] for j in model.g)<=df['MaxDisp'][i]
    )

In [30]:
model.profit = Objective(expr=sum(sum(model.F[i,j] for i in model.c)*df2['Precio'][j] for j in model.g)-
sum(sum(model.F[i,j] for j in model.g)*df['Coste'][i] for i in model.c), sense=maximize)

In [31]:
results = SolverFactory('glpk').solve(model)
results.write() 
model.write('Resultados/Ej19.lp', io_options={'symbolic_solver_labels': True})
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 11502.7777777778
  Upper bound: 11502.7777777778
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 13
  Number of nonzeros: 37
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.050115108489990234
# ----------------------------------------------------------
#   Solution Information
# -----------